In [1]:
Packages <- c("dplyr",  "RMySQL",  "tidyr", "lubridate")
invisible(suppressPackageStartupMessages(lapply(Packages, library, character.only = TRUE)))
setwd('/local/home/katrinac/parentage')

load("~/parentage/r_data/sampled_area_each_year.RData")
load("~/parentage/r_data/anems_visited_by_year.RData")
load("~/parentage/r_data/total_sampling_across_years.RData")

#download.file(url = "https://raw.githubusercontent.com/pinskylab/genomics/master/data/known_issues.csv", destfile = "~/parentage/r_data/known_issues.csv")
issues <- read.csv("~/parentage/r_data/known_issues.csv", header=T, stringsAsFactors = F)
#download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Data_from_database/fish_db.RData?raw=true", destfile = "~/parentage/r_data/dives_db.RData")
load("~/parentage/r_data/fish_db.RData")
#download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Data_from_database/anem_db.RData?raw=true", destfile = "~/parentage/r_data/anem_db.RData")
load("~/parentage/r_data/anem_db.RData")
#download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Data_from_database/dives_db.RData?raw=true", destfile = "~/parentage/r_data/dives_db.RData")
load("~/parentage/r_data/dives_db.RData")
#download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Data_from_database/fish_db.RData?raw=true", destfile = "~/parentage/r_data/dives_db.RData")
load("~/parentage/r_data/fish_db.RData")
load("~/parentage/r_data/gps_db.RData")
#download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/anems_tagged.RData", destfile = "~/parentage/r_data/anems_tagged.RData")
load("~/parentage/r_data/anems_tagged.RData")
#download.file(url = "https://github.com/pinskylab/genomics/blob/master/data/fish-obs.RData?raw=true", destfile = "~/parentage/r_data/fish-obs.RData")
fish_obs <- readRDS("~/parentage/r_data/fish-obs.RData") 
#download.file(url = "https://raw.githubusercontent.com/pinskylab/db_backups/master/ligation_1-7-20.csv", destfile = "~/parentage/r_data/ligation_db.csv")
lig_db <- read.csv("~/parentage/r_data/ligation_db.csv", header=T, stringsAsFactors = F)
#download.file(url = "https://raw.githubusercontent.com/pinskylab/db_backups/master/digest_1-7-20.csv", destfile = "~/parentage/r_data/digest_db.csv")
dig_db <- read.csv("~/parentage/r_data/digest_db.csv", header=T, stringsAsFactors = F)
#download.file(url = "https://raw.githubusercontent.com/pinskylab/db_backups/master/extraction_1-7-20.csv", destfile = "~/parentage/r_data/extraction_db.csv")
ext_db <- read.csv("~/parentage/r_data/extraction_db.csv", header=T, stringsAsFactors = F)
"%!in%" <- function(x,table) match(x,table, nomatch = 0) == 0


In [2]:
nrow(fish_obs)

[1] 3819

In [3]:
dat_gen <- read.table("/local/home/katrinac/parentage/colony2/20190422_894loci/input/seq33-03_norecap.gen", colClasses= "character",  quote="", header=TRUE, row.names=NULL, sep=" ")
loci_to_remove <- read.table("/local/home/katrinac/parentage/colony2/20190422_894loci/input/20190523_894_plink.prune.out", quote="", header=T, stringsAsFactors = F)

In [92]:
dim(dat_gen) #697 loci total
#dim(loci_to_keep)


#do this after filtering for only loci not in LD and maf 0.05
#names(dat_gen) <- gsub("dDocent_Contig_" , "",  names(dat_gen))  
#names(dat_gen) <- gsub("\\d_" , "",  names(dat_gen))



#find and drop the loci that have three alleles

#flip the data frame so you can use filter

#dat_gen$ligation_id

row.names(dat_gen) <-dat_gen$ligation_id

#dat_gen$ligation_id <- NULL


##could use this code to split columns rather than doing it as in terminal20180517 --> cSplit(mydf, grep("Allele", names(mydf)), "", stripWhite = FALSE)


dat_gen_t <- as.data.frame(t(dat_gen), stringsAsFactors = F)#look for loci appended with 2 for the 3 allele loci
#dat_gen_t$ligation_id <- rownames(dat_gen_t)

dat_gen_t$locus <- rownames(dat_gen_t)
#head(dat_gen_t)

ligation_ids <- dat_gen %>%
    select(ligation_id)

no_ld <- dat_gen_t %>%
  filter(locus=="ligation_id" | locus %!in% loci_to_remove$locus)
#
dim(no_ld)
#
#
##drop the columns with a 03 value
#
dat_gen_t_beta <- no_ld %>% mutate(third_allele = rowSums(no_ld=="03"))
#
sum(dat_gen_t_beta$third_allele != 0) #one locus of the 697 is in LD
#
#head(dat_gen_t_beta)
#
dat_gen_beta2 <- dat_gen_t_beta %>% filter(third_allele == 0)
#
##head(dat_gen_beta2)
#
col <- dat_gen_beta2$locus
dat_gen_beta2$locus <- NULL
dat_gen_beta2$third_allele <- NULL
#
dat_gen <- t(dat_gen_beta2)
#
#
dat_gen <- as.data.frame(dat_gen, stringsAsFactors = F, row.names = NULL, col.names = col)
#
names(dat_gen) <- col
#
##dat_gen$ligation_id <- row.names(dat_gen)
row.names(dat_gen) <- NULL
#
dim(dat_gen)
#
#dat_gen2 <- dat_gen %>%
#    filter(ligation_id %in% dat_gen_FORIDS$ligation_id)

[1] 2406 2680

[1] 2680 2407

[1] 0

[1] 2406 2680

In [ ]:
#correct fish obs

In [5]:
#2895-2680
#2680/2
#215/2

In [4]:
#dim(dat_gen)

In [53]:
#write.table(dat_gen, file="/local/home/katrinac/parentage/colony2/20190523_1340loci/input/20190523_1340_loci_seq33-03_noLD.txt", col.names=T, quote=FALSE, row.names=FALSE)



In [4]:
#load the sequencing data already modified
dat_gen <- read.table("/local/home/katrinac/parentage/colony2/20190523_1340loci/input/20190523_1340_loci_seq33-03_noLD.txt", colClasses= "character",  quote="", header=TRUE, row.names=NULL)
#dat_gen[is.na(dat_gen)] <- 00

In [5]:
#from Allison, just putting all the meta data together (Constants_database_common_functions.R)
##### Match up other relevant info (site, date, fish_indiv, etc.) to fish in the clownfish table
# Pull out year and month into a separate column in dives_db
dives_db_processed <- dives_db %>%
  mutate(year = as.integer(substring(date,1,4))) %>%
  mutate(month = as.integer(substring(date,6,7))) %>%
  mutate(dive_date = date(date))

# Pull all APCL caught or otherwise in the clownfish table
allfish_fish <- fish_db %>%
  select(fish_table_id, anem_table_id, fish_spp, sample_id, anem_table_id, recap, tag_id, color, sex, size, fish_obs_time, fish_notes) %>%
  filter(fish_spp == 'APCL') %>%
  mutate(size = as.numeric(size))  # make the size numeric (rather than chr) so can do means and such

# and their corresponding anemones
allfish_anems <- anem_db %>%
  select(anem_table_id, dive_table_id, anem_obs, anem_id, old_anem_id, anem_notes) %>%
  filter(anem_table_id %in% allfish_fish$anem_table_id)

# and the corresponding dive info
allfish_dives <- dives_db_processed %>%
  select(dive_table_id, dive_type, date, year, month, site, gps, dive_notes) %>%
  filter(dive_table_id %in% allfish_anems$dive_table_id) 

# join together
allfish_caught <- left_join(allfish_fish, allfish_anems, by="anem_table_id")
allfish_caught <- left_join(allfish_caught, allfish_dives, by="dive_table_id")

# add in the gen_ids and fish_indiv (now in a separate gen_id table) - gen_id only comes in the time the fish was sequenced, not at all captures
allfish_caught <- left_join(allfish_caught, (fish_obs %>% select(fish_table_id, gen_id, fish_indiv)), by = "fish_table_id")

#get sample ids for ligation ids
ext_db <- ext_db %>%
    select(extraction_id, sample_id)
dig_db <- dig_db %>%
    select(extraction_id, digest_id)
lig_db <- lig_db %>%
    select(ligation_id, digest_id)
ext_dig <- left_join(ext_db, dig_db, by="extraction_id")
dig_lig <- left_join(ext_dig, lig_db, by="digest_id") %>% 
    filter(!is.na(ligation_id))
lig_samp <- dig_lig %>%
    select(ligation_id, sample_id) 

In [6]:
fish_in_gen_lig <- dat_gen %>% #pull of the ligation_ids of fish in dat_gen, and remove those in the issues table
    select(ligation_id) %>%
    filter(ligation_id %!in% issues$ligation_id) #5 fish in issues table, these are dropped
nrow(dat_gen) -nrow(fish_in_gen_lig) 

fish_in_gen_samp <- lig_samp %>% 
    filter(ligation_id %in%dat_gen$ligation_id) %>%
    distinct(ligation_id, sample_id, .keep_all = T) #all of the ligation_ids in fish_in_gen_samp are in fish_in_gen_lig. Are the extra 7 sample_ids just multiples for regenotyped fish? YES see the following when this line is commented out: test %>% group_by(ligation_id, sample_id) %>% summarise(n=n()) %>% filter(n >1)

nrow(fish_in_gen_lig)- nrow(fish_in_gen_samp) #should be 0

sampled_fish <- left_join(allfish_caught, lig_samp,  by="sample_id") %>%
    select(fish_indiv, year, size, color, sex, gen_id, ligation_id, sample_id) #%>%
    #filter(!is.na(sample_id))

fish_meta <- sampled_fish %>%#join with the rest of the data
    select(fish_indiv, year, size, color, sex, gen_id, ligation_id, sample_id)
###join with genetic data
dat_gen_meta <-  left_join(dat_gen, fish_meta, by="ligation_id")

[1] 0

[1] 0

In [20]:
#checking for fish that may be sorted incorrectly because their observations aren't connected by a gen_id
#(allfish_caught %>% filter(!is.na(tag_id) & size < 6)%>% select(sample_id, tag_id, gen_id, color, sex, size, date, site, fish_indiv) %>% mutate(fish_indiv=as.numeric(fish_indiv)) %>% filter(fish_indiv < 5000))
##what are the known observations of this fish?
#id_eval <- 2633
#allfish_caught %>% filter(fish_indiv==id_eval) %>% select(sample_id, tag_id, gen_id, color, sex, size, date, site, fish_indiv)
##is it correctly sorted now?
#offs_annual %>% filter(fish_indiv==id_eval) #made June 4, 2020 after changing the parent/offspring assignment loop to allow a previous observation of a fish to be considered as sampled even though that earlier ligation id was not in the genepop file because it wasn't the best ligation/genotype data for that fish
#parents %>% filter(fish_indiv==id_eval)  #made June 4, 2020 after changing the parent/offspring assignment loop to allow a previous observation of a fish to be considered as sampled even though that earlier ligation id was not in the genepop file because it wasn't the best ligation/genotype data for that fish


In [8]:
nrow(fish_meta) #should be 8478

[1] 8478

In [15]:
dim(dat_gen_meta)# should be 2406 x 2687

[1] 2406 2687

### Make a gen file for all adults from every year.

In [7]:
years <- as.matrix(c(2012, 2013, 2014, 2015, 2016, 2017, 2018)) #years to evaluate
indiv_ids <- as.matrix(dat_gen_meta %>% #gen_ids to sort into parent/offspring files
                     select(fish_indiv) %>% 
                     distinct(fish_indiv))

parents <- as.data.frame(matrix(nrow=0, ncol=8))  #copy previous year parent file to current parent file by starting this file out of the loop and building on it in the loop. mutate a column with the year it will be a part of, and sort through that as "for 2013, grab everything in the parent file >=2013" after the loop
colnames(parents) <- names(fish_meta)

offs_annual <- as.data.frame(matrix(nrow=0, ncol=8))
colnames(offs_annual) <- names(fish_meta)
    
for(i in 1:nrow(years)){
    
    year_eval <- years[i]
    
    for(k in 1:nrow(indiv_ids)){ 
        
        indiv_id_eval <- indiv_ids[k]
        
        meta_eval <- fish_meta %>% #this will be the parents
            filter(fish_indiv == indiv_id_eval & year == year_eval) %>% #year == year_eval, taking this out in testing with AD, add it back in later if you want and sort for year later in the loop
            filter(size >= 6 | sex=="M" | sex=="F")  %>%
            #filter(sex != "J" ) %>%
            #filter(fish_indiv == indiv_id_eval) %>% #& ligation_id %in% best_genos$ligation_id) #commented out 06/01/2020
            distinct(fish_indiv, .keep_all = T) #06/01/2020, shouldn't need this unless I fish was sampled multiple times in a year, which could happen if they were caught on a recap dive, but then there wouldn't be a sample id for them. and even if there was it wouldn't matter because the observation would be the same size/year probably so distinct is works to get rid of that if it's happening
        
        if(indiv_id_eval %in% meta_eval$fish_indiv){#then it's a parent
       
            if(indiv_id_eval %!in% parents$fish_indiv){ #if it's not already in the parent file, add it #maybe change this to elseif
                
                parents <-  bind_rows(meta_eval, parents)  

                
            }
            
#then it's an offspring
        }else{
            
          
             if(indiv_id_eval %!in% offs_annual$fish_indiv){ #if it's not already in the offspring file, add it
                    
                    add_to_offspring <- fish_meta %>%
                            filter(fish_indiv == indiv_id_eval & year==year_eval)%>% #& ligation_id %in% best_genos$ligation_id)
                            distinct(fish_indiv, .keep_all = T)
                 offs_annual <- bind_rows(offs_annual, add_to_offspring) 
                 
        
            }
        }

    }
}
parents <- parents #%>%
    #filter(!is.na(gen_id))
offs_annual <- offs_annual #%>%
    #filter(!is.na(gen_id))


In [8]:
nrow(parents)
nrow(offs_annual) #any fish with a weird tail color here (YP or O) has been corrected by Michelle, as indicated in the "fish_correction" column.
nrow(parents)+nrow(offs_annual)#should be 2625

[1] 1729

[1] 791

[1] 2520

In [18]:
#print and upload the histogram of parent/offspring sizes, the total fish captured in each year, total offspring in the offspring file for each year, total in the parents of each year, and the number of each tail color in the parent and offspring file so everyone can look on github, total number of parents in parentage.

In [27]:
##double check total number
#test_n <- bind_rows(offs_annual, parents) %>% 
#    distinct(fish_indiv, .keep_all = T)
#nrow(test_n) #2405
#fish_meta %>% distinct(fish_indiv, .keep_all = T) %>% filter(fish_indiv < 5000 & fish_indiv %!in% test_n$fish_indiv) #SHOULD BE EMPTY
#
##png(file="~/parentage/colony2/20190523_1340loci/input/size_distribution_parents.png")
#hist(parents$size, main="size distribution of fish in parent files (all <6cm are YP/F or O/M)")#looks good, but make sure the fish smaller than 6 in the parents. YES, see test below
##dev.off()
##png(file="~/parentage/colony2/20190523_1340loci/input/size_distribution_offspring.png")
#hist(offs_annual$size, main="size distribution of fish in offspring files") #looks good
##dev.off()
#test <- parents %>% filter(size <6)
#(test$sex)
#
##table of summaries
#
##how many fish from each year?
#n_samples <- fish_meta %>% filter(!is.na(sample_id)) %>% group_by(year) %>% summarise(n_observed=n()) 
#sum(n_samples$n_observed)
#n_samples
##write.table(n_samples, file="~/parentage/colony2/20190523_1340loci/input/annual_number_fish_geno.txt", col.names = T, quote=F, row.names = F)
#
##what are the number of fish of each sex in parents and offspring file?
#par_sex <- parents %>% group_by(sex) %>% summarise(n=n()) 
##write.table(par_sex, file="~/parentage/colony2/20190523_1340loci/input/parent_nsex.txt", col.names = T, quote=F, row.names = F)
#offs_sex <- offs_annual %>% group_by(sex) %>% summarise(n=n()) 
##write.table(offs_sex, file="~/parentage/colony2/20190523_1340loci/input/offspring_nsex.txt", col.names = T, quote=F, row.names = F)
#
#

In [ ]:
#change NA genotypes to 0 for Colony
#parents_gen[is.na(parents_gen)] <- 00
#offs_gen[is.na(offs_gen)] <- 00

In [20]:
dat_gen_meta %>% filter(is.na(ligation_id))

all_par13 %>% filter(is.na(ligation_id))

all_par13_gen[301,]

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”

ligation_id,dDocent_Contig_11_59.01,dDocent_Contig_11_59.02,dDocent_Contig_11_87.01,dDocent_Contig_11_87.02,dDocent_Contig_11_93.01,dDocent_Contig_11_93.02,dDocent_Contig_11_117.01,dDocent_Contig_11_117.02,dDocent_Contig_11_126.01,...,dDocent_Contig_233984_13.02,dDocent_Contig_233984_104.01,dDocent_Contig_233984_104.02,fish_indiv,year,size,color,sex,gen_id,sample_id


fish_indiv,year,size,color,sex,gen_id,ligation_id,sample_id


,ligation_id,dDocent_Contig_11_59.01,dDocent_Contig_11_59.02,dDocent_Contig_11_87.01,dDocent_Contig_11_87.02,dDocent_Contig_11_93.01,dDocent_Contig_11_93.02,dDocent_Contig_11_117.01,dDocent_Contig_11_117.02,dDocent_Contig_11_126.01,...,dDocent_Contig_207387_131.01,dDocent_Contig_207387_131.02,dDocent_Contig_207875_52.01,dDocent_Contig_207875_52.02,dDocent_Contig_226526_58.01,dDocent_Contig_226526_58.02,dDocent_Contig_233984_13.01,dDocent_Contig_233984_13.02,dDocent_Contig_233984_104.01,dDocent_Contig_233984_104.02
301,L2356,01,02,01,01,01,01,01,01,01,...,01,02,01,01,02,02,01,02,01,02


In [14]:
nrow(all)

,ligation_id,dDocent_Contig_11_59.01,dDocent_Contig_11_59.02,dDocent_Contig_11_87.01,dDocent_Contig_11_87.02,dDocent_Contig_11_93.01,dDocent_Contig_11_93.02,dDocent_Contig_11_117.01,dDocent_Contig_11_117.02,dDocent_Contig_11_126.01,...,dDocent_Contig_233984_13.02,dDocent_Contig_233984_104.01,dDocent_Contig_233984_104.02,fish_indiv,year,size,color,sex,gen_id,sample_id
2401,L5622,01,02,01,01,01,01,01,01,01,...,02,01,02,1649,2018,9.3,O,M,1649,APCL18_728
2402,L5637,01,02,01,01,01,01,01,01,01,...,02,01,02,2267,2018,7.9,YP,F,2267,APCL18_719
2403,L5638,01,01,01,01,01,02,01,02,01,...,02,01,02,2200,2018,6.6,O,M,2200,APCL18_729
2404,L5639,01,02,01,01,01,01,01,01,01,...,02,01,02,1648,2018,8.4,YP,F,1648,APCL18_730
2405,L5640,01,01,01,01,01,01,01,01,01,...,02,01,02,1647,2018,9.9,YP,F,1647,APCL18_731
2406,L5655,01,01,01,01,01,01,01,01,01,...,02,01,02,889,2012,8.1,O,M,889,APCL12_255


In [20]:
#sort into Male/Female/Offspring and annual files
#2012

all_offs12 <- offs_annual %>%
    filter(year == 2012)  
all_offs12_gen <- dat_gen_meta %>%
    filter(fish_indiv %in% all_offs12$fish_indiv) %>%
    distinct(fish_indiv, .keep_all = T) %>% 
    select(-fish_indiv, -year, -size, -color, -sex, -gen_id, -sample_id)

all_par12 <- parents %>%
    filter(year==2012 & fish_indiv %!in% all_offs12$fish_indiv)
all_par12_gen <- dat_gen_meta %>%
    filter(fish_indiv %in% all_par12$fish_indiv) %>%
    distinct(fish_indiv, .keep_all = T)%>% 
    select(-fish_indiv, -year, -size, -color, -sex, -gen_id, -sample_id)



#2013

all_offs13 <- offs_annual %>%
    filter(year==2013)
all_offs13_gen <- dat_gen_meta %>%
    filter(fish_indiv %in% all_offs13$fish_indiv) %>%
    distinct(fish_indiv, .keep_all = T)%>% 
    select(-fish_indiv, -year, -size, -color, -sex, -gen_id, -sample_id)

all_par13 <- parents %>%
    filter(year %in% c(2012, 2013) & fish_indiv %!in% all_offs13$fish_indiv)
all_par13_gen <- dat_gen_meta %>%
    filter(fish_indiv %in% all_par13$fish_indiv) %>%
    distinct(fish_indiv, .keep_all = T)%>% 
    select(-fish_indiv, -year, -size, -color, -sex, -gen_id, -sample_id)



#2014
all_offs14 <- offs_annual %>%
    filter(year==2014)
all_offs14_gen <- dat_gen_meta %>%
    filter(fish_indiv %in% all_offs14$fish_indiv) %>%
    distinct(fish_indiv, .keep_all = T)%>% 
    select(-fish_indiv, -year, -size, -color, -sex, -gen_id, -sample_id)


all_par14 <- parents %>%
    filter(year %in% c(2012, 2013, 2014) & fish_indiv %!in% all_offs14$fish_indiv) 
all_par14_gen <- dat_gen_meta %>%
    filter(fish_indiv %in% all_par14$fish_indiv) %>%
    distinct(fish_indiv, .keep_all = T)%>% 
    select(-fish_indiv, -year, -size, -color, -sex, -gen_id, -sample_id)



##2015
all_offs15 <- offs_annual %>%
    filter(year==2015)
all_offs15_gen <- dat_gen_meta %>%
    filter(fish_indiv %in% all_offs15$fish_indiv) %>%
    distinct(fish_indiv, .keep_all = T)%>% 
    select(-fish_indiv, -year, -size, -color, -sex, -gen_id, -sample_id)

all_par15 <- parents %>%
    filter(year %in% c(2012, 2013, 2014, 2015) & fish_indiv %!in% all_offs15$fish_indiv) 
all_par15_gen <- dat_gen_meta %>%
    filter(fish_indiv %in% all_par15$fish_indiv) %>%
    distinct(fish_indiv, .keep_all = T)%>% 
    select(-fish_indiv, -year, -size, -color, -sex, -gen_id, -sample_id)



##2016
all_offs16 <- offs_annual %>%
    filter(year==2016)
all_offs16_gen <- dat_gen_meta %>%
    filter(fish_indiv %in% all_offs16$fish_indiv) %>%
    distinct(fish_indiv, .keep_all = T)%>% 
    select(-fish_indiv, -year, -size, -color, -sex, -gen_id, -sample_id)


all_par16 <- parents %>%
    filter(year %in% c(2012, 2013, 2014, 2015, 2016) & fish_indiv %!in% all_offs16$fish_indiv) 
all_par16_gen <- dat_gen_meta %>%
    filter(fish_indiv %in% all_par16$fish_indiv) %>%
    distinct(fish_indiv, .keep_all = T)%>% 
    select(-fish_indiv, -year, -size, -color, -sex, -gen_id, -sample_id)



#2017
all_offs17 <- offs_annual %>%
    filter(year==2017) 
all_offs17_gen <- dat_gen_meta %>%
    filter(fish_indiv %in% all_offs17$fish_indiv) %>%
    distinct(fish_indiv, .keep_all = T)%>% 
    select(-fish_indiv, -year, -size, -color, -sex, -gen_id, -sample_id)


all_par17 <- parents %>%
    filter(year %in% c(2012, 2013, 2014, 2015, 2016, 2017) & fish_indiv %!in% all_offs17$fish_indiv)  
all_par17_gen <- dat_gen_meta %>%
    filter(fish_indiv %in% all_par17$fish_indiv) %>%
    distinct(fish_indiv, .keep_all = T)%>% 
    select(-fish_indiv, -year, -size, -color, -sex, -gen_id, -sample_id)



#2018
all_offs18 <- offs_annual %>%
    filter(year==2018)
all_offs18_gen <- dat_gen_meta %>%
    filter(fish_indiv %in% all_offs18$fish_indiv)%>% 
    select(-fish_indiv, -year, -size, -color, -sex, -gen_id, -sample_id)


all_par18 <- parents %>%
    filter(year %in% c(2012, 2013, 2014, 2015, 2016, 2017, 2018) & fish_indiv %!in% all_offs18$fish_indiv) %>%
    distinct(fish_indiv, .keep_all = T)
all_par18_gen <- dat_gen_meta %>%
    filter(fish_indiv %in% all_par18$fish_indiv) %>%
    distinct(fish_indiv, .keep_all = T)%>% 
    select(-fish_indiv, -year, -size, -color, -sex, -gen_id, -sample_id)



#pooled analysis
pooled <- bind_rows(parents, offs_annual) %>%
    distinct(fish_indiv, .keep_all = T)
pooled_gen <- dat_gen_meta %>%
    filter(fish_indiv %in% pooled$fish_indiv)%>% 
    distinct(fish_indiv, .keep_all = T) %>%
    select(-fish_indiv, -year, -size, -color, -sex, -gen_id, -sample_id) 
#nrow(test_n %>% filter(fish_indiv %!in% pooled$fish_indiv)) #check, should be 0



In [12]:
offs_gen <- dat_gen_meta %>%
    filter(fish_indiv %in% offs_annual$fish_indiv)%>% 
    distinct(fish_indiv, .keep_all = T) %>%
    select(-fish_indiv, -year, -size, -color, -sex, -gen_id, -sample_id) 
write.table(offs_gen, file="~/parentage/colony2/20190523_1340loci/input/20200605_offs_geneticsAll.txt", col.names=FALSE, quote=FALSE, row.names=FALSE )


In [28]:
tail(all_par18_gen)

,ligation_id,dDocent_Contig_11_59.01,dDocent_Contig_11_59.02,dDocent_Contig_11_87.01,dDocent_Contig_11_87.02,dDocent_Contig_11_89.01,dDocent_Contig_11_89.02,dDocent_Contig_11_93.01,dDocent_Contig_11_93.02,dDocent_Contig_11_117.01,...,dDocent_Contig_207875_52.01,dDocent_Contig_207875_52.02,dDocent_Contig_226526_58.01,dDocent_Contig_226526_58.02,dDocent_Contig_233984_13.01,dDocent_Contig_233984_13.02,dDocent_Contig_233984_81.01,dDocent_Contig_233984_81.02,dDocent_Contig_233984_104.01,dDocent_Contig_233984_104.02
1724,L5622,01,02,01,01,03,03,01,01,01,...,01,02,02,02,01,02,02,03,01,02
1725,L5637,01,02,01,01,03,03,01,01,01,...,02,02,02,02,01,02,02,03,01,02
1726,L5638,01,01,01,01,01,03,01,02,01,...,01,02,02,02,01,02,02,03,01,02
1727,L5639,01,02,01,01,03,02,01,01,01,...,01,02,01,02,01,02,02,03,01,02
1728,L5640,01,01,01,01,03,03,01,01,01,...,01,02,01,02,01,02,02,03,01,02
1729,L5655,01,01,01,01,01,03,01,01,01,...,01,01,02,02,01,02,02,03,01,02


In [30]:
dim(all_par13_gen)

[1]  372 2680

In [46]:
#fish_obs %>% 
#    filter(!is.na(gen_id)) %>% 
#    filter(!is.na(tag_id))%>% 
#    group_by(fish_indiv) %>% 
#    filter(n()>1) %>% 
#    ungroup() %>% 
#    #distinct(gen_id, .keep_all = T) %>% 
#    group_by(gen_id) %>% 
#    filter(n()>1)
#
#

In [31]:
input_summary <- as.data.frame(matrix(nrow=8, ncol=3))
col <- c("year", "n_offspring", "n_parents")
colnames(input_summary) <- col
#
input_summary$year <- c("2012", "2013", "2014", "2015", "2016", "2017", "2018", "all_years")
input_summary$n_offspring <- c(nrow(all_offs12_gen), nrow(all_offs13_gen), nrow(all_offs14_gen), nrow(all_offs15_gen), nrow(all_offs16_gen), nrow(all_offs17_gen), nrow(all_offs18_gen), nrow(offs_annual))
input_summary$n_parents <- c(nrow(all_par12_gen), nrow(all_par13_gen), nrow(all_par14_gen), nrow(all_par15_gen), nrow(all_par16_gen), nrow(all_par17_gen), nrow(all_par18_gen), nrow(parents))
#write.table(input_summary, file="~/parentage/colony2/20190523_1340loci/input/input_summary.txt", col.names=T, quote=FALSE, row.names=FALSE )


In [22]:
write.table(all_offs12_gen, file="~/parentage/colony2/20190523_1340loci/input/20200605_offs_genetics12.txt", col.names=FALSE, quote=FALSE, row.names=FALSE )
write.table(all_par12_gen, file="~/parentage/colony2/20190523_1340loci/input/20200605_par_genetics12.txt", col.names=FALSE, quote=FALSE, row.names=FALSE )



In [43]:
head(dat_gen_meta)

ligation_id,dDocent_Contig_11_59.01,dDocent_Contig_11_59.02,dDocent_Contig_11_87.01,dDocent_Contig_11_87.02,dDocent_Contig_11_93.01,dDocent_Contig_11_93.02,dDocent_Contig_11_117.01,dDocent_Contig_11_117.02,dDocent_Contig_11_126.01,...,dDocent_Contig_233984_13.02,dDocent_Contig_233984_104.01,dDocent_Contig_233984_104.02,fish_indiv,year,size,color,sex,gen_id,sample_id
L0255,01,02,01,01,01,01,01,01,01,...,02,01,02,558,2013,8.2,O,M,558,APCL13_220
L0256,01,02,01,01,01,01,01,01,01,...,02,01,02,339,2013,3.0,YR,J,339,APCL13_221
L0257,01,02,01,01,01,01,01,01,01,...,02,02,02,1809,2013,6.9,YP,F,1809,APCL13_222
L0258,01,01,01,01,02,01,01,01,01,...,02,01,02,185,2013,2.5,Y,J,185,APCL13_223
L0259,01,01,01,01,01,02,01,01,01,...,02,01,02,320,2013,4.6,W,J,320,APCL13_224
L0260,01,01,01,01,01,01,01,01,01,...,02,01,02,457,2013,7.9,O,M,457,APCL13_225


In [32]:
input_summary
nrow(pooled_gen)#should be 2405
sum(input_summary$n_offspring)-nrow(offs_annual)
nrow(parents)
nrow(all_offs15 %>% filter(fish_indiv %in% all_offs16$fish_indiv))#not sure why there's the same number of offspring in 2015 & 2016, but it's coincidence. This is 0

year,n_offspring,n_parents
2012,63,157
2013,150,372
2014,181,536
2015,111,850
2016,111,1232
2017,130,1453
2018,45,1729
all_years,791,1729


[1] 2405

[1] 791

[1] 1729

[1] 0

In [160]:
#write the offspring and parent files for the N_captured values in kernel_fit_file_prep.ipynb
write.table(parents, file="~/parentage/colony2/20190523_1340loci/input/all_parents_corrected.txt", col.names=T, quote=FALSE, row.names=FALSE )
write.table(offs_annual, file="~/parentage/colony2/20190523_1340loci/input/all_offspring_corrected.txt", col.names=T, quote=FALSE, row.names=FALSE )


In [45]:
all_par13_gen %>%
    filter(ligation_id =="L2987")

ligation_id,dDocent_Contig_11_59.01,dDocent_Contig_11_59.02,dDocent_Contig_11_87.01,dDocent_Contig_11_87.02,dDocent_Contig_11_93.01,dDocent_Contig_11_93.02,dDocent_Contig_11_117.01,dDocent_Contig_11_117.02,dDocent_Contig_11_126.01,...,dDocent_Contig_207387_131.01,dDocent_Contig_207387_131.02,dDocent_Contig_207875_52.01,dDocent_Contig_207875_52.02,dDocent_Contig_226526_58.01,dDocent_Contig_226526_58.02,dDocent_Contig_233984_13.01,dDocent_Contig_233984_13.02,dDocent_Contig_233984_104.01,dDocent_Contig_233984_104.02
L2987,00,00,00,00,00,00,00,00,00,...,01,02,01,02,01,02,02,02,02,02


In [44]:
#put together into 1 text file
space <- matrix(nrow=1, ncol=ncol(dat_gen))
par13_mat <- as.matrix(all_par13_gen)


offs13_mat <- as.matrix(all_offs13_gen)

full13 <- rbind(offs13_mat, space, par13_mat, space, par13_mat)

write.table(full13, file="~/parentage/colony2/20190523_1340loci/input/full_genetics13.txt", col.names=FALSE, quote=FALSE, row.names=FALSE )
#write.table(pooled_gen, file="~/parentage/colony2/20190523_1340loci/input/full_genetics_pooled.txt", col.names=FALSE, quote=FALSE, row.names=FALSE )



### make full file with marker input

In [240]:
## make a markers file

dup <- parents_gen %>% select(-ligation_id,-year, -contains(".02"))

#make loci only number for colony
names(dup) <- gsub("dDocent_Contig_" , "",  names(dup))  
names(dup) <- gsub("\\d_" , "",  names(dup))
names(dup) <- gsub(".01" , "",  names(dup))  


locus_names <- as.matrix(colnames(dup))
length(locus_names) #should be 696
error_matrix <- matrix(nrow=4, ncol=894)
error_matrix[1,] <- locus_names[1:894]
error_matrix[2,] <- 0
error_matrix[3,] <- 0
error_matrix[4,] <- 0.026
head(error_matrix)
#write.table(error_matrix, file="/local/home/katrinac/parentage/colony2/20190422_894loci/input/894_loci_error_mat.txt", col.names=FALSE, row.names=FALSE, quote=FALSE)



[1] 894

159,1030,1048,3177,31166,3346,33107,437,43159,5115,⋯,1995764,1995772,2888,20215127,20738131,2078752,2265258,2339813,23398104,23398147
0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
0.026,0.026,0.026,0.026,0.026,0.026,0.026,0.026,0.026,0.026,⋯,0.026,0.026,0.026,0.026,0.026,0.026,0.026,0.026,0.026,0.026


In [241]:
#proportion sampled in each time frame for colony "prob of parent in parent file" input
total_sampling_across_years$total_area_method <- as.character(total_sampling_across_years$total_area_method)
total_sampling_across_years$total_anems_method <- as.character(total_sampling_across_years$total_anems_method)

In [85]:
total_sampling_across_years %>% filter(total_area_method=="anems tidied" &total_anems_method=="metal tags")

total_anems_method,time_frame,total_prop_hab_sampled,total_area_method
metal tags,2012,0.1592760,anems tidied
metal tags,2012-2013,0.2357466,anems tidied
metal tags,2012-2014,0.2775264,anems tidied
metal tags,2012-2015,0.3061086,anems tidied
metal tags,2012-2016,0.3286878,anems tidied
metal tags,2012-2017,0.3538462,anems tidied
metal tags,2012-2018,0.4005171,anems tidied


In [383]:
input_summary #2018 n_parents should be the full number of parents, 1403
sum(input_summary$n_offspring) #should be 1080 with 694 loci data, 798 with 894 loci


year,n_offspring,n_parents
2012,63,157
2013,150,372
2014,181,536
2015,229,849
2016,105,1230
2017,130,1447
2018,45,1722
all_years,903,1722


[1] 1806

# DONE, everything below is testing.